In [60]:
import extract
import numpy as np
from sklearn.feature_extraction import FeatureHasher
import pandas as pd
import re

In [2]:
d = extract.get_data()
d = extract.clean_data(d)

/home/mark/Math/bondproject/venv/local/lib/python2.7/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/mark/Math/bondproject/venv/local/lib/python2.7/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


In [3]:
d.head()

,tradeid,cusip,amount,price,yield,tradedate,tradetype,name,state,coupon,maturity,issuesize,issuetype,issuesource,bidcount,rtg
0,317.0,45471AKG6,5000.0,99.507,4.037,0.0,Sale_to_Customer,indiana st fin auth-a,Indiana,4.00,6635.0,88930000.0,REVENUE BONDS,"HLTH, HOSP, NURSHOME REV.",NaN,6
1,318.0,45471AKG6,30000.0,99.410,4.045,0.0,Sale_to_Customer,indiana st fin auth-a,Indiana,4.00,6635.0,88930000.0,REVENUE BONDS,"HLTH, HOSP, NURSHOME REV.",NaN,6
2,319.0,45471AKG6,30000.0,99.290,4.054,0.0,Purchase_from_Customer,indiana st fin auth-a,Indiana,4.00,6635.0,88930000.0,REVENUE BONDS,"HLTH, HOSP, NURSHOME REV.",NaN,6
3,333.0,45471AMY5,15000.0,103.145,3.717,0.0,Sale_to_Customer,in fin auth-major,Indiana,4.25,6423.0,9520000.0,REVENUE BONDS,"HLTH, HOSP, NURSHOME REV.",NaN,9
4,378.0,4550572N8,150000.0,114.082,2.060,0.0,Sale_to_Customer,in fin auth-ref-a,Indiana,5.00,2894.0,22530000.0,REVENUE BONDS,WATER REVENUE,NaN,1


In [5]:
d.name[:5]

0    indiana st fin auth-a
1    indiana st fin auth-a
2    indiana st fin auth-a
3        in fin auth-major
4        in fin auth-ref-a
Name: name, dtype: object

In [6]:
state_names = list(d.state.unique()) + ['massachusets']
state_names = [s.lower() for s in state_names]
state_names.sort()

In [10]:
#remove state_names from string name if they do occur
#state names always seem to occur at beginning of name
def state_strip(s):
    for state in state_names:
        if state in s:
            s = ''.join(s.split(state))
    return s.strip()

In [14]:
state_abbr = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
	"HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
	"MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
	"NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
	"SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
state_abbr = [s.lower() for s in state_abbr]

def state_abbr_filter(s):
	if s in state_abbr:
		return ''

	for code in state_abbr:
		if " " + code + " " in s:
			return ''.join(s.split(" " + code + " "))

		if " " + code in s[-3:]:
			return s[:-3]

		if code + " " in s[:3]:
			return s[3:]
		
	return s

In [15]:
L = d.name.apply(state_strip).apply(state_abbr_filter).str.split(r'[\-/ ]', expand = True)

In [35]:
mask = L.applymap(lambda x: x is None)
cols = L.columns[mask.any()]
for col in L[cols]:
    L.loc[mask[col], col] = ""
    
L.head()

,0,1,2,3,4,5,6,7,8
0,st,fin,auth,a,,,,,
1,st,fin,auth,a,,,,,
2,st,fin,auth,a,,,,,
3,fin,auth,major,,,,,,
4,fin,auth,ref,a,,,,,


In [36]:
fh = FeatureHasher(input_type='string',non_negative=True)

In [37]:
L_t = fh.transform(L.values)

In [38]:
L_t

<35456x1048576 sparse matrix of type '<type 'numpy.float64'>'
	with 159442 stored elements in Compressed Sparse Row format>

In [39]:
type(L_t)

scipy.sparse.csr.csr_matrix

In [40]:
L_t[0,0]

5.0

In [41]:
L.shape

(35456, 9)

In [48]:
S.unique().size

2429

In [49]:
d.name.values[:10]

array(['indiana st fin auth-a', 'indiana st fin auth-a',
       'indiana st fin auth-a', 'in fin auth-major', 'in fin auth-ref-a',
       'maryland-second ser b', 'maryland-second ser b',
       'maryland-second ser b', 'maryland-second ser b', 'maryland st'], dtype=object)

In [61]:
pattern = re.compile(r'[\-/ ]')
def clean_text(s):
    s = state_strip(s)
    s = state_abbr_filter(s)
    return re.split(pattern, s)

In [62]:
X = [clean_text(s) for s in d.name.values]

In [63]:
X[:5]

[['st', 'fin', 'auth', 'a'],
 ['st', 'fin', 'auth', 'a'],
 ['st', 'fin', 'auth', 'a'],
 ['fin', 'auth', 'major'],
 ['fin', 'auth', 'ref', 'a']]

In [64]:
len(X)

35456

In [65]:
fh = FeatureHasher(input_type='string',non_negative=True)

In [66]:
X_t = fh.transform(X)

In [67]:
type(X_t)

scipy.sparse.csr.csr_matrix

In [68]:
X_t.shape

(35456, 1048576)

In [83]:
y = [X_t[j,:].dot(X_t[0,:].T) for j in range(100)]

In [85]:
X_t[0,:].shape

(1, 1048576)

In [87]:
fh.get_params()

{'dtype': numpy.float64,
 'input_type': 'string',
 'n_features': 1048576,
 'non_negative': True}

In [88]:
2**31

2147483648

1048576